In [1]:
import os
import sys
import sqlite3
from dotenv import load_dotenv
import importlib.metadata

# Project root
project_root = os.path.abspath("..")
sys.path.append(project_root)

# Load environment variables
load_dotenv(os.path.join(project_root, ".env"))

True

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

scope = (
    "playlist-read-private "
    "playlist-read-collaborative "
    "playlist-modify-private "
    "playlist-modify-public "
    "user-library-read "
    "user-read-recently-played"
)

cache_path = os.path.join(project_root, ".spotify_cache")

sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        scope=scope,
        cache_path=cache_path
    )
)

print("Spotify authenticated.")
print("Current user:", sp.current_user()["id"])
print("Spotipy version:", importlib.metadata.version("spotipy"))

Spotify authenticated.
Current user: 21ogollmd4ljapwluvwxe3stq
Spotipy version: 2.25.2


In [3]:
from core.database import get_connection, create_tables, get_latest_added_at
from core.ingestion import sync_new_tracks, sync_playlists

db_path = os.path.join(project_root, "music_agent.db")

conn = get_connection(db_path)
create_tables(conn)

print("Database connected.")

Database connected.


In [4]:
print("Syncing saved tracks...")
new_tracks = sync_new_tracks(sp, conn, get_latest_added_at)
print("New tracks inserted:", new_tracks)

Syncing saved tracks...
New tracks inserted: 0


In [5]:
print("Syncing playlists...")
playlist_stats = sync_playlists(sp, conn)

print("Playlists synced:", playlist_stats["playlists_synced"])
print("Playlist tracks synced:", playlist_stats["playlist_tracks_synced"])

Syncing playlists...
Playlists synced: 5
Playlist tracks synced: 59


In [ ]:
from core.database import get_connection, create_tables, get_latest_added_at
from core.ingestion import sync_new_tracks

db_path = os.path.join(project_root, "music_agent.db")

conn = get_connection(db_path)
create_tables(conn)

print("Database connected.")
print("SQLite version:", sqlite3.sqlite_version)

cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("Tables:", cursor.fetchall())

# Sync saved tracks
new_tracks = sync_new_tracks(sp, conn, get_latest_added_at)
print("New tracks inserted:", new_tracks)

cursor.execute("SELECT COUNT(*) FROM tracks;")
print("Total tracks:", cursor.fetchone()[0])

In [ ]:
from core.db_session import DatabaseSession
from core.repository import Repository
from core.behavior import ingest_recently_played

db_session = DatabaseSession(db_path)
repo = Repository(db_session)

# Ingest recently played behavior (ya usa repo)
inserted = ingest_recently_played(sp, repo, limit=50)
print("Inserted real play events:", inserted)

print("Total play_history rows:", repo.get_recently_played_track_names(limit=1000))

In [ ]:
from core.ingestion import sync_playlists

stats = sync_playlists(sp, conn)
print(stats)

In [ ]:
from core.ingestion import sync_playlists

playlist_stats = sync_playlists(sp, conn)
print("Playlist sync stats:", playlist_stats)

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

In [ ]:
from core.graph.builder import build_music_graph

# Asegúrate de tener llm inicializado antes
graph = build_music_graph(repo, sp, llm)

print("Graph compiled successfully.")

In [ ]:
from core.graph.builder import build_music_graph

In [ ]:
graph = build_music_graph(repo, sp, llm)
print(graph)

In [ ]:
state = {
    "user_input": "Create a playlist with my top played songs this month",
    "strategy": None,
    "result_tracks": None,
    "error": None,
    "needs_clarification": False,
    "clarification_message": None,
    "confirmed": False,
    "intent": "build"
}

from core.graph.nodes import strategy_node

test = strategy_node(state, llm)
test["strategy"]

In [ ]:
state = {
    "user_input": "Create a playlist with my top played songs this month",
    "strategy": None,
    "result_tracks": None,
    "error": None,
    "needs_clarification": False,
    "clarification_message": None,
    "confirmed": False,
    "intent": "build"
}

from core.graph.nodes import strategy_node

test = strategy_node(state, llm)
test["strategy"]

In [ ]:
result = graph.invoke({
    "user_input": "Create a playlist with my top played songs this month",
    "strategy": None,
    "result_tracks": None,
    "error": None,
    "needs_clarification": False,
    "clarification_message": None,
    "confirmed": False,
    "intent": ""
})

result

In [ ]:
result["confirmed"] = True
final = graph.invoke(result)
final

In [ ]:
#1
result = graph.invoke({
    "user_input": "Create a playlist with my top played songs this month and my recently added tracks",
    "strategy": None,
    "result_tracks": None,
    "error": None,
    "needs_clarification": False,
    "clarification_message": None,
    "confirmed": False,
    "intent": ""
})

result

In [ ]:
#2
result = graph.invoke({
    "user_input": "Create a playlist with all my songs from Drake",
    "strategy": None,
    "result_tracks": None,
    "error": None,
    "needs_clarification": False,
    "clarification_message": None,
    "confirmed": False,
    "intent": ""
})
result


In [ ]:
#3
result = graph.invoke({
    "user_input": "Create a playlist with my songs from the album Graduation",
    "strategy": None,
    "result_tracks": None,
    "error": None,
    "needs_clarification": False,
    "clarification_message": None,
    "confirmed": False,
    "intent": ""
})
result

In [ ]:
#4..9
result = graph.invoke({
    "user_input": "Make it better",
    "strategy": None,
    "result_tracks": None,
    "error": None,
    "needs_clarification": False,
    "clarification_message": None,
    "confirmed": False,
    "intent": ""
})
result



In [ ]:
#5
result = graph.invoke({
    "user_input": "Create a playlist with 10 of my most played songs",
    "strategy": None,
    "result_tracks": None,
    "error": None,
    "needs_clarification": False,
    "clarification_message": None,
    "confirmed": False,
    "intent": ""
})
result


In [ ]:
#6
result = graph.invoke({
    "user_input": "Change the name of Energia Total to This is a test",
    "strategy": None,
    "result_tracks": None,
    "error": None,
    "needs_clarification": False,
    "clarification_message": None,
    "confirmed": False,
    "intent": ""
})
result


In [ ]:
result["confirmed"] = True
final = graph.invoke(result)
final

In [ ]:
#7
result = graph.invoke({
    "user_input": "Remove Drake songs from the playlist Locura",
    "strategy": None,
    "result_tracks": None,
    "error": None,
    "needs_clarification": False,
    "clarification_message": None,
    "confirmed": False,
    "intent": ""
})
result

In [ ]:
#8
result = graph.invoke({
    "user_input": "Add my recently added songs and my top played songs to the playlist Locura",
    "strategy": None,
    "result_tracks": None,
    "error": None,
    "needs_clarification": False,
    "clarification_message": None,
    "confirmed": False,
    "intent": ""
})
result

In [ ]:
#9
result = graph.invoke({
    "user_input": "Rename the playlist Locura to Energia Total",
    "strategy": None,
    "result_tracks": None,
    "error": None,
    "needs_clarification": False,
    "clarification_message": None,
    "confirmed": False,
    "intent": ""
})
result


In [ ]:
result["confirmed"] = True
final = graph.invoke(result)
final

In [ ]:
from IPython.display import Image



In [ ]:
Image(graph.get_graph().draw_mermaid_png())

In [ ]:
from core.graph.builder import build_music_graph
from session.manager import SessionManager

graph = build_music_graph(repo, sp, llm)

session = SessionManager(graph)

session.handle("Create a playlist with Drake")
session.handle("Yes")
session.handle("Add my most played songs")

In [ ]:
session.handle("Do we have music from Dezko?")
session.handle("Create a playlist with that")

In [ ]:
session.handle("Build me an airplane")

In [ ]:
session.handle("Do we have music from Dezko?")

In [ ]:
session.handle("Create a playlist with Drake")

In [ ]:
session.handle("Do we have music from Dezko?")
session.handle("Create a playlist with that")

In [ ]:
session.handle("Yes")

In [ ]:
session.handle("Do we have music from Dezko?")
session.handle("Create a playlist with that")
session.handle("Yes")

In [ ]:
print("---- TEST 1: Basic Build ----")
r1 = session.handle("Create a playlist with Drake")
print(r1["clarification_message"])

r2 = session.handle("Yes")
print(r2["clarification_message"])

In [ ]:
print("---- TEST 2: Info → Build with reference ----")
r1 = session.handle("Do we have music from Dezko?")
print(r1["clarification_message"])

r2 = session.handle("Create a playlist with that")
print(r2["clarification_message"])

r3 = session.handle("Yes")
print(r3["clarification_message"])

In [ ]:
print("---- TEST 3: Unknown ----")
r = session.handle("Build me a spaceship")
print(r["intent"], r["clarification_message"])

In [ ]:
print(session.context.phase)

In [ ]:
print("---- TEST 4: Modify Add ----")

r1 = session.handle("Create a playlist with Drake")
session.handle("Yes")

r2 = session.handle("Add my most played songs to AI Generated Playlist")
print(r2["clarification_message"])

r3 = session.handle("Yes")
print(r3["clarification_message"])

In [ ]:
print("---- TEST 5: Orphan Yes ----")
r = session.handle("Yes")
print(r)

In [ ]:
session.context.last_interaction_ts = session.context.last_interaction_ts.replace(year=2000)
r = session.handle("Create a playlist with Drake")
print(session.context.phase)

In [ ]:
from core.graph.builder import build_music_graph
from session.manager import SessionManager

graph = build_music_graph(repo, sp, llm)

session = SessionManager(graph)

In [ ]:
r1 = session.handle("Create a playlist called Pepito with songs from Rihanna and Drake")
print(r1["clarification_message"])
r2 = session.handle("Yes")
print(r2["clarification_message"])

In [ ]:
r2 = session.handle("Create a playlist with reggaeton music")
print(r2["clarification_message"])

In [ ]:
r3 = session.handle("Rename my playlist AI Generated Playlist to  Summer vibes")
print(r3["clarification_message"])

In [ ]:
r4 = session.handle("Add songs from Dezko songs to the playlist AI Generated Playlist")
print(r4["clarification_message"])

In [ ]:
r5 = session.handle("what playlist do I have?")
print(r5["clarification_message"])

In [ ]:
r6 = session.handle("Do i have music from Drake in any playlist?")
print(r6["clarification_message"])

In [ ]:
r7 = session.handle("Do i have music from Adele in my library?")
print(r7["clarification_message"])

In [ ]:
r8 = session.handle("Do I have music from Adele?")
print(r8["clarification_message"])
r9 = session.handle("yes")
print(r9["clarification_message"])

In [ ]:
r9 = session.handle("Create a playlist called Late Night Drive with songs from The Weeknd, Drake and Future")
print(r9["clarification_message"])
r9 = session.handle("yes")
print(r9["clarification_message"])

In [ ]:
r9 = session.handle("yes")

In [ ]:
r10 = session.handle("what playlist do I have?")
print(r10["clarification_message"])
r11 = session.handle("yes?")
print(r11["clarification_message"])

In [ ]:
r = session.handle("Do I have music from Drake in any playlist?")
print(r["strategy"])
print(r["clarification_message"])

In [ ]:
r = session.handle("Do I have music from Drake?")
print(r["clarification_message"])

In [ ]:
spotify_playlists = sp.current_user_playlists(limit=50)["items"]
print("Spotify playlists:", len(spotify_playlists))

print("DB playlists:", repo.count_playlists())

In [ ]:
spotify_names = [p["name"] for p in spotify_playlists]
db_names = repo.get_all_playlists()

print("Missing in DB:")
for name in spotify_names:
    if name not in db_names:
        print("-", name)

In [ ]:
drake_library = repo.get_tracks_by_artist("Drake")
print("Drake tracks in library:", len(drake_library))

In [ ]:
drake_in_playlists = repo.count_artist_tracks_in_playlists("Drake")
print("Drake tracks in playlists (DB):", drake_in_playlists)

In [ ]:
# Ver tracks reales de Drake en una playlist específica
playlist_name = "Heavy"  # cambia por una que sepas que tiene Drake
playlist_tracks = repo.get_playlist_tracks(
    repo.conn.execute("SELECT playlist_id FROM playlists WHERE name = ?", (playlist_name,)).fetchone()[0]
)

print("Tracks in playlist:", len(playlist_tracks))

drake_ids = set(repo.get_tracks_by_artist("Drake"))
intersection = [tid for tid in playlist_tracks if tid in drake_ids]

print("Drake tracks inside playlist (intersection):", len(intersection))

In [ ]:
cursor = repo.conn.cursor()
cursor.execute("SELECT COUNT(*) FROM playlist_tracks;")
print("Total playlist_tracks rows:", cursor.fetchone()[0])

In [ ]:
print("Tracks in DB:", repo.conn.execute("SELECT COUNT(*) FROM tracks;").fetchone()[0])
print("Playlist tracks in DB:", repo.conn.execute("SELECT COUNT(*) FROM playlist_tracks;").fetchone()[0])

In [ ]:
repo.count_artist_tracks_in_playlists("Drake")